In [ ]:

import matplotlib.pyplot as plt # type: ignore
import pandas as pd# type: ignore
from sklearn.model_selection import train_test_split# type: ignore
from sklearn.linear_model import LinearRegression# type: ignore
from sklearn.metrics import mean_squared_error# type: ignore
from sklearn.model_selection import train_test_split# type: ignore
import statsmodels.api as sm# type: ignore
import numpy as np



In [ ]:
df =   pd.read_csv('products.csv')

In [ ]:

print(df)